In [1]:
# default values hard coded
# placed at top for convience to edit 

ScheduleChangeWeight_Default = 1
MinAlloc_Default = 0.01
OverTimePenalty_Default = 0.01
ExternalWeight_Default = 10



expected_headers_projects = ['Name', 'External?', 'Value', 'Clearance', 'Location', "Project Management"]
expected_headers_people = ["Name",	"Clearance",	"Location",	"Project Management"]

In [2]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import math

# Title: Capstone Project: Schedule optimization application for L3Haris
# Author: Ewan Hassall, Tyler Burpe, Michael Gruver, Samantha Anderson, and Kenneth Dang
# Date: 04/21/2024
# ISEN 460 
# L3Haris

# version 2.0 notes 
#: clearance / location issue solved with introduction of clearance_secret and clearance_top_secret variables
# header check now uses .lower() function to ignore capitalizaion
# added comments




current_schedule_exists = 0
# Create a tkinter root window (hidden)
root = tk.Tk()
root.withdraw()

# Ask the user to select the Excel file
file_path = filedialog.askopenfilename(filetypes=[('Excel files', '*.xlsx')])
if not file_path:
    print("No file selected.")
else:
    # Read the data from the Excel file
    with pd.ExcelFile(file_path) as xls:
        if 'People' in xls.sheet_names:
            people_df = pd.read_excel(xls, sheet_name='People')
            actual_headers_people = people_df.columns[:len(expected_headers_people)].tolist()
            
             # Check if the actual headers match the expected headers
            # uses .lower() to ignore caps
            actual_headers_people = [skill.lower() for skill in actual_headers_people]
            expected_headers_people = [skill.lower() for skill in expected_headers_people]

            if actual_headers_people != expected_headers_people:
        
                raise ValueError("Excel column headers in people do not match the expected headers")
                exit()
            
            if 'Name' in people_df.columns:
                name_column = 'Name'
            else:
                name_column = people_df.columns[0]  # First column if 'Name' is not a column header

            # Extract names and check for duplicates
            names = people_df[name_column]
            if names.duplicated().any():
                duplicates = names[names.duplicated()].unique()
                raise ValueError(f"Duplicate names found: {duplicates}. Please remove duplicates and try again.")
                exit()
           

            actual_skills_people = people_df.columns[len(expected_headers_people):].tolist()
            
        else:
            # if the people sheet is missing throw error
            print("ERROR: Missing People data")
            raise ValueError("ERROR: Missing Projects data")
            exit()

        # checks for projects sheet existing
        if 'Projects' in xls.sheet_names:
            projects_df = pd.read_excel(xls, sheet_name='Projects')
            actual_headers_projects = projects_df.columns[:len(expected_headers_projects)].tolist()
            
             # Check if the actual headers match the expected headers
             # .lower() to ignore caps
            actual_headers_projects = [skill.lower() for skill in actual_headers_projects]
            expected_headers_projects = [skill.lower() for skill in expected_headers_projects]

            if actual_headers_projects != expected_headers_projects:
                raise ValueError("Excel column headers in projects do not match the expected headers")
                exit()

             # Check if 'Project Name' column exists, otherwise assume the first column contains project names

            if 'Name' in projects_df.columns:
                project_name_column = 'Name'
            else:
                project_name_column = projects_df.columns[0]  # First column if 'Project Name' is not a column header

            # Extract project names and check for duplicates
            project_names = projects_df[project_name_column]
            if project_names.duplicated().any():
                duplicates = project_names[project_names.duplicated()].unique()
                raise ValueError(f"Duplicate project names found: {duplicates}. Please remove duplicates and try again.")


            actual_skills_projects = projects_df.columns[len(expected_headers_projects):].tolist()
            
            actual_skills_projects = [skill.lower() for skill in actual_skills_projects]
            
            actual_skills_people = [skill.lower() for skill in actual_skills_people]
            if (actual_skills_projects != actual_skills_people):
                print(actual_skills_projects)
                print(actual_skills_people)
                print(len(expected_headers_projects))
                print(len(expected_headers_people))
                raise ValueError("Names of skills and Skill requirnments do not match, check order and spelling")
                exit()
        else:
            print("ERROR: Missing Projects data")
            raise ValueError("ERROR: Missing Projects data")
            exit()
        
      

        if 'Parameters' in xls.sheet_names:
            params_df = pd.read_excel(xls, sheet_name='Parameters', index_col=0)
            try:
                ScheduleChangeWeight = params_df.at['ScheduleChangeWeight', 'Value']
            except KeyError:
                print("Warning: 'ScheduleChangeWeight' not found. Using default value:", ScheduleChangeWeight_Default )
                ScheduleChangeWeight = ScheduleChangeWeight_Default
            try:
                MinAlloc = params_df.at['MinAlloc', 'Value']
            except KeyError:
                print("Warning: 'MinAlloc' not found. Using default value:", MinAlloc_Default)
                MinAlloc = MinAlloc_Default
            try:
                OverTimePenalty = params_df.at['OverTimePenalty', 'Value']
            except KeyError:
                print("Warning: 'OverTimePenalty' not found. Using default value:", OverTimePenalty_Default)
                OverTimePenalty = OverTimePenalty_Default
            try:
                ExternalWeight = params_df.at['ExternalWeight', 'Value']
            except KeyError:
                print("Warning: 'ExternalWeight' not found. Using default value:", ExternalWeight_Default)
                ExternalWeight = ExternalWeight_Default
        else:
            print("Parameters sheet not found. Using default values.")
            ScheduleChangeWeight = ScheduleChangeWeight_Default
            MinAlloc = MinAlloc_Default
            OverTimePenalty = OverTimePenalty_Default
            ExternalWeight = ExternalWeight_Default

        if 'Current Schedule' in xls.sheet_names:
            Current_Schedule_df = pd.read_excel(xls, sheet_name='Current Schedule')
            current_schedule = Current_Schedule_df.to_dict(orient='records')
            current_schedule_exists = 1
        else:
            print("WARNING: No Current Schedule Sheet found")
            
        
        
    # Recreate the lists
    people = people_df.to_dict(orient='records')
    projects = projects_df.to_dict(orient='records')
   

# Recreate the lists
people = people_df.to_dict(orient='records')
projects = projects_df.to_dict(orient='records')


print("People:")
for person in people:
    print(person)

print("\nProjects:")
for project in projects:
    print(project)


# Extracting values and assigning them to variables
# Read the parameters from the Excel file


print("Variables set from Excel sheet:")
print("ScheduleChangeWeight =", ScheduleChangeWeight)
print("MinAlloc =", MinAlloc)
print("OverTimePenalty =", OverTimePenalty)
print("ExternalWeight =", ExternalWeight)

People:
{'Name': '1Skill', 'Clearance': 0, 'Location': 0, 'Project Management': 0, '1Skill': 1, '2Skill': 0, '3Skill': 0, '4Skill': 0, '5Skill': 0, '6Skill': 0, '7Skill': 0, '8Skill': 0, '9Skill': 0, '10Skill': 0, '11Skill': 0, '12Skill': 0}
{'Name': '2Skill', 'Clearance': 1, 'Location': 0, 'Project Management': 0, '1Skill': 0, '2Skill': 1, '3Skill': 0, '4Skill': 0, '5Skill': 0, '6Skill': 0, '7Skill': 0, '8Skill': 0, '9Skill': 0, '10Skill': 0, '11Skill': 0, '12Skill': 0}
{'Name': '3Skill', 'Clearance': 2, 'Location': 0, 'Project Management': 0, '1Skill': 0, '2Skill': 0, '3Skill': 1, '4Skill': 0, '5Skill': 0, '6Skill': 0, '7Skill': 0, '8Skill': 0, '9Skill': 0, '10Skill': 0, '11Skill': 0, '12Skill': 0}
{'Name': '4Skill', 'Clearance': 0, 'Location': 1, 'Project Management': 0, '1Skill': 0, '2Skill': 0, '3Skill': 0, '4Skill': 1, '5Skill': 0, '6Skill': 0, '7Skill': 0, '8Skill': 0, '9Skill': 0, '10Skill': 0, '11Skill': 0, '12Skill': 0}
{'Name': '5Skill', 'Clearance': 1, 'Location': 1, 'Proje

In [3]:

# Function to check for NaN values in a series
def check_for_string_nan(list, list_name):
    for i in list:
        if(math.isnan(i)):

            raise ValueError(f"The string 'nan' found in {list_name}.")
            exit()
            
    
    
# this section of code processes our larger data sets containing all info about people and projects
# into catagorized data lists such as people_clearance which can be fed into the ampl code
# we also Check each list for the string nan values while doing this



# Define the skill data for each person
people_data = people
project_data = projects
people_skills = {} 
people_names = [] # assigns each person a number ID (1,2,..., n) which is required for ample
people_names_string = {} # actual string values for names
project_names_string = {}# actual string values for names
for i, person in enumerate(people_data):
    skills = list(person.values())[3:]  # Exclude the first 3 values which is the name, location, clerance
    people_skills[i] = skills
    check_for_string_nan(skills, "people skills data")
    
    people_names.append(i)
print(people_skills)
projectskillsrequirnments = {}
project_names = [] # assigns each project  a number ID (1,2,..., n) which is required for ample
for i, project in enumerate(project_data):
    requirnments = list(project.values())[5:]  # Exclude the first 5 values which is the name, value, external, location and  Clearance
    projectskillsrequirnments[i] = requirnments
    check_for_string_nan(requirnments, "projects skills requirnments data")
    project_names.append(i)

# create list of the values for each project (as in their worth or utility)
values = {}
for i, project in enumerate(project_data):
    value = list(project.values())[2:3]  
    values[i] = value
    check_for_string_nan(value, "project values")
values = [val for sublist in values.values() for val in sublist]


# makes the list for external vs internal projects
external = {}
for i, project in enumerate(project_data):
    value = list(project.values())[1:2]  
    external[i] = value
    check_for_string_nan(value, "external values")
external = [val for sublist in external.values() for val in sublist]

# updates project values based on if the project is external
# external projects are given extra value so that they will be prioritized frist by LP
for i in range(len(values)):
    values[i] = values[i] + values[i] * external[i] * ExternalWeight


# creates clearance lsits
# staff_clearance = actualy clearance value imported from excel
# clearance_secret is binary, 1 if person i can work on secret and 0 if not
# clearance_top_secret is binary, 1 if person i can work on top secret and 0 if not
# clearance_secret and top_secret are infered from _clearance
# this is done to solve location/clearance issues where it becomes possible that 
# someone can have top_secret but not have secret clearance
Staff_Clearance = {}
Staff_Clearance_secret = {}
Staff_Clearance_top_secret = {}
for i, project in enumerate(people_data):
    Clearance = list(project.values())[1:2]  
    Staff_Clearance[i] = Clearance
    Staff_Clearance_secret[i] = Clearance
    Staff_Clearance_top_secret[i] = Clearance
    
    check_for_string_nan(Clearance, "staff Clearance values")

# i dont know why i need the following lines of code  for _secret and _top_secret when we are literally just about
# to  redefine them in the next lines of code but if remove this line the program crashes 

Staff_Clearance_secret = [val for sublist in Staff_Clearance.values() for val in sublist]
Staff_Clearance_top_secret = [val for sublist in Staff_Clearance.values() for val in sublist]
Staff_Clearance = [val for sublist in Staff_Clearance.values() for val in sublist]

# defines _secret and _top_secret correctly
for i in range(len(Staff_Clearance)):
    if Staff_Clearance[i] > 0:
        Staff_Clearance_secret[i] = 1
    else:
        Staff_Clearance_secret[i] = 0
    if Staff_Clearance[i] > 1:  
        Staff_Clearance_top_secret[i] = 1
    else:
        Staff_Clearance_top_secret[i] = 0

ProjectClearance = {}
Project_Clearance_secret = {}
Project_Clearance_top_secret = {}
for i, project in enumerate(project_data):
    Clearance = list(project.values())[3:4]  
    ProjectClearance[i] = Clearance
    Project_Clearance_secret[i] = Clearance
    Project_Clearance_top_secret[i] = Clearance
    
    check_for_string_nan(Clearance, "project Clearance values")

# i dont know why i need the following lines of code  for _secret and _top_secret when we are literally just about
# to  redefine them in the next lines of code but if remove this line the program crashes 

Project_Clearance_secret = [val for sublist in ProjectClearance.values() for val in sublist]
Project_Clearance_top_secret = [val for sublist in ProjectClearance.values() for val in sublist]
ProjectClearance = [val for sublist in ProjectClearance.values() for val in sublist]

# defines _secret and _top_secret correctly
for i in range(len(ProjectClearance)):
    if ProjectClearance[i] == 1:
        Project_Clearance_secret[i] = 1
    else:
        Project_Clearance_secret[i] = 0
    if ProjectClearance[i] == 2:  
        Project_Clearance_top_secret[i] = 1
    else:
        Project_Clearance_top_secret[i] = 0
# creates location data
locations_Projects = {}
for i, project in enumerate(project_data):
    location = list(project.values())[4:5]  # Exclude the first value which is the name, value, location Clearance
    locations_Projects[i] = location
    check_for_string_nan(location, "project location values")
locations = [val for sublist in locations_Projects.values() for val in sublist]
print(locations_Projects)

# creates location data
locations_Staff = {}
for i, project in enumerate(people_data):
    location_Staff = list(project.values())[2:3]  
    locations_Staff[i] = location_Staff
    check_for_string_nan(location_Staff, "staff location values")
locations_Staff = [val for sublist in locations_Staff.values() for val in sublist]
print(locations_Staff)



# this section of codes adjusts staff clearance based on the location they are at
# currently this code does not adjust project clearance based on location

# remote = 0, can only work on unclassified (0)
# palm bay = 1, can work on any classification (0,1,2)
# northen va = 2, can only work on unclassified (0) and top secret (2)
# cliften nj = 3, can only work on unclassified (0) and secret (1)

print(len(Staff_Clearance) == len(Staff_Clearance_secret))
for i in range(len(Staff_Clearance)):
    # remote only work unclassified
    if locations_Staff[i] == 0:
        Staff_Clearance_secret[i] =  0
        Staff_Clearance_top_secret[i] = 0
    # palm bay can do anything loc = 1
    
    # northen va doesnt do secret loc = 2
    if locations_Staff[i] == 2:
        print(i)
        print(Staff_Clearance_secret[i])
        Staff_Clearance_secret[i] =  0
        
   
   # cliften NJ: loc = 3 can only work on unclassified (0) and secret (1)
    if locations_Staff[i] == 3:
        Staff_Clearance_top_secret[i] = 0

# line of code would be used to update project clearance if desired
#for j in range(len(ProjectClearance)):
#    if locations_Projects[j] == 0:
 #       ProjectClearance[j] = ProjectClearance[j]

print("clearance levels")
print(Project_Clearance_secret)
print(Project_Clearance_top_secret)
print(Staff_Clearance_secret)
print(Staff_Clearance_top_secret)

{0: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 4: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 5: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 6: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 7: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 8: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 10: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 11: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}
{0: [0], 1: [0], 2: [0], 3: [0], 4: [0], 5: [0], 6: [0], 7: [0], 8: [0], 9: [0], 10: [0], 11: [0]}
[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]
True
6
0
7
1
8
1
clearance levels
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]


In [4]:


# saves the len of our lists, need this info for ampl
num_people = len(people_skills)
num_projects = len(projectskillsrequirnments)
num_skills = len(people_skills[0])

skill_names = []
people_names = []
project_names = []
print(num_people)

# creates ID's for each person,project and skill (skill 1, skill 2,... skill n; person 1, person 2,.. person n),
# this is nesseary for ampl code
for  i in range(1,num_people+1):
    people_names.append(i)
for  j in range(1,num_projects+1):
    project_names.append(j)
for  k in range(1,num_skills+1):
    skill_names.append(k)

print(project_names)

12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [5]:
# schedule change Penalty needs two matrixes
# first asks if this column is applicable for a penetly it is binary
# second determines the values we want to stay consistant to
# Convert people and projects to sets for faster lookup

#current_names = {assignment['person'] for assignment in current_schedule}

# Create a matrix with zeros
matrix = pd.DataFrame(0, index=[person['Name'] for person in people], columns=[project['Name'] for project in projects], dtype=float)
matrixBinary = pd.DataFrame(0, index=[person['Name'] for person in people], columns=[project['Name'] for project in projects], dtype=float)

if current_schedule_exists == 1: # dont bother if the current schedule doesnt exist
    for assignment in current_schedule: 
        person = assignment.pop('Person')  # Remove the 'Person' key and get its value
        if person in matrix.index:
            for project, value in assignment.items():
                if project in matrix.columns and value != 0:
                    matrix.at[person, project] = value
                    matrixBinary.at[person, project] = 1

# Print the matrix
print(matrix)
matrix_1D = matrix.values.flatten().tolist() # ampl requires a 1d matrix not a 2d
matrixBinary_1D = matrixBinary.values.flatten().tolist()
# Print the one-dimensional list
print(matrix)

         1Skill  2Skill  3Skill  4Skill  5Skill  6Skill  7Skill  8Skill  \
1Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
6Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
7Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
8Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9Skill      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
10Skill     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
11Skill     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12Skill     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

         9Skill  10Skill

In [6]:
from amplpy import AMPL, Environment

# Create an AMPL instance
ampl = AMPL()

# start by defining all 0D paramaters
# 0d params must come first because they define the dimentions of other params
ampl.eval("""
        param num_people;
        param num_projects;
        param num_skills;
        param ScheduleChangeWeight;
        param MinAlloc;
        param OverTimePenalty;
          """)

ampl.param['num_people'] = num_people
ampl.param['num_projects'] = num_projects
ampl.param['num_skills'] = num_skills

ampl.param['MinAlloc'] = MinAlloc
ampl.param['OverTimePenalty'] = OverTimePenalty
ampl.param['ScheduleChangeWeight'] = ScheduleChangeWeight

print(num_skills)
print(num_projects)
# Define dimentional params, sets and the decision variables
# params and sets are datas we import from excel
# vars are the decision variables
ampl.eval("""
    set PEOPLE within 1..num_people;
    set PROJECTS within 1..num_projects;
    set SKILLS within 1..num_skills;
   
    param ProjectRequirnments{1..num_projects, 1..num_skills};
          
    param PeopleSkills{1..num_people, 1..num_skills};
          
    param ProjectValues {1..num_projects};
    param Project_Clearance_secret {1..num_projects};
    param Project_Clearance_top_secret {1..num_projects};
    param Staff_Clearance_secret {1..num_people};
    param Staff_Clearance_top_secret {1..num_people};
    param current_schedule_matrix{1..num_people, 1..num_projects};
    param current_schedule_matrixBinary{1..num_people, 1..num_projects};
        
 
    var PercentAllocated{PEOPLE, PROJECTS, SKILLS} <= 2 >= 0 ;
    
    var IfAllocated{PEOPLE, PROJECTS, SKILLS} binary;
    var amountOfOvertimeUsed{PEOPLE} >= 0 <= 0.2;
    var Completed{PROJECTS} binary;
    var ScheduleChangePenalty;
          
""")

#

13
12


In [7]:


#print(people_skills)
print(people_names)
ampl.set['PEOPLE'] = people_names
ampl.set['PROJECTS'] = project_names
ampl.set['SKILLS'] = skill_names
ampl.param['Project_Clearance_secret'] = Project_Clearance_secret
ampl.param['Project_Clearance_top_secret'] = Project_Clearance_top_secret
ampl.param['Staff_Clearance_secret'] = Staff_Clearance_secret
ampl.param['Staff_Clearance_top_secret'] = Staff_Clearance_top_secret
ampl.param['current_schedule_matrix'] = matrix_1D
ampl.param['current_schedule_matrixBinary'] = matrixBinary_1D
#p#rint(num_people)
#print(num_skills)
#print(people_skills)
#learn_values = [i *10+j for i in range(1, 9+1) for j in range(1, 4+1)]
#print(learn_values)
people_skills_1D = [val for sublist in people_skills.values() for val in sublist]

#print(people_skills_1D)
ampl.getParameter('PeopleSkills').setValues(people_skills_1D)


project_skills_1D = [val for sublist in projectskillsrequirnments.values() for val in sublist]

print(project_skills_1D)

ampl.getParameter('ProjectRequirnments').setValues(project_skills_1D)
ampl.eval('display PeopleSkills;')

ampl.getParameter('ProjectValues').setValues(values)
print(Staff_Clearance)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
PeopleSkills [*,*] (tr)
:    1   2   3   4   5   6   7   8   9  10  11  12    :=
1    0   0   0   0   0   0   0   0   0   0   0   0
2    1   0   0   0   0   0   0   0   0   0   0   0
3    0   1   0   0   0   0   0   0   0   0   0   0
4    0   0   1   0   0   0   0   0   0   0   0   0
5    0   0   0   1   0   0   0   0   0   0   0   0
6    0   0   0   0   1   0   0   0   0   0   0   0
7    0   0   0   0   0   1   0   0   0   0   0   0
8    0   0   0   0   0   0   1   0   0   0   0   0
9 

In [8]:
 #Define the objective function
ampl.eval("""
  
            # score = (Number of projects assigned) * (value of each project) 
          # additionally score punishes allocations in general (dont use people unless you have to)
          # punishes overtime (dont use overtime unless you have to)
          # punishes schedule changes (dont change the schedule from the previous month unless you have to)
    maximize score:
        
        sum {j in 1..num_projects} Completed[j] * (ProjectValues[j] ) 
        - sum {i in 1..num_people, j in 1..num_projects, k in 1..num_skills} MinAlloc * PercentAllocated[i,j,k] 
        - sum {i in 1..num_people} OverTimePenalty * amountOfOvertimeUsed[i]
        - sum {i in 1..num_people} ScheduleChangeWeight * ScheduleChangePenalty
          ;
""")
#ampl.param['MinAlloc'] = MinAlloc
#ampl.param['OverTimePenalty'] = OverTimePenalty
#ampl.param['ScheduleChangeWeight'] = ScheduleChangeWeight

In [9]:
# Define constraints
ampl.eval("""

# Cant declare a project "completed" unless all project requirnments have been met by
# allocating sufficent skills to the project
subject to Project_Completeness_Constraint{ j in 1..num_projects, k in 1..num_skills}:
    sum {i in 1..num_people} (PercentAllocated[i,j,k] * PeopleSkills[i,k])  >=  (ProjectRequirnments[j,k] * Completed[j]);

# scheduleChangePenalty
subject to Schedule_CHange_Penalty:
    sum {i in 1..num_people,j in 1..num_projects} current_schedule_matrixBinary[i,j] * abs(current_schedule_matrix[i,j] - sum{k in 1..num_skills} (PercentAllocated[i,j,k]))  <= ScheduleChangePenalty;

          
# do not assign less than 33 percent of someones time to a project
subject to AllocationConstraint1{i in PEOPLE, j in 1..num_projects, k in 2..num_skills}:
     PercentAllocated[i,j,k] >= 0.33 * IfAllocated[i,j,k];
          
# project management specificically
subject to ProjectManagement_constraint{i in PEOPLE, j in 1..num_projects, k in 1..1}:
     PercentAllocated[i,j,k] >= 0.03 * IfAllocated[i,j,k];
          
# do not assign over 100 percent of someones time unless you use overtime
subject to AllocationConstraint2{i in PEOPLE, j in 1..num_projects, k in 1..num_skills}:
     PercentAllocated[i,j,k] <= (1 + amountOfOvertimeUsed[i]) * IfAllocated[i,j,k];

# dont let someones cumulative time spent be over 100%
subject to People_MaxWorkingTimeConstraint{i in PEOPLE}:
    sum {j in 1..num_projects, k in 1..num_skills} PercentAllocated[i,j,k] <= 1+ amountOfOvertimeUsed[i];

# if project clearance = 2 then need class_clearance_top_secret needs to = 1 dont caare about secret
# if project clearance =1 then staff_clearance_secret needs to = 1 dont care about top secret

subject to ClearanceConstraint_secret{i in PEOPLE, j in 1..num_projects, k in 1..num_skills}:
    (Project_Clearance_secret[j] - Staff_Clearance_secret[i]) * PercentAllocated[i,j,k] <= 0;
          
subject to ClearanceConstraint_top_secret{i in PEOPLE, j in 1..num_projects, k in 1..num_skills}:
    (Project_Clearance_top_secret[j] - Staff_Clearance_top_secret[i]) * PercentAllocated[i,j,k] <= 0;        

""")
#ampl.getVariable("PercentAllocated"). 
#ampl.get_variable(""Assignment 02 MTDE 333-633 (2).docx"")

In [10]:
#ampl.getVariable("PercentAllocated").setValues([1])
#ampl.getVariable("PercentAllocated").

In [11]:

# Solve the model
ampl.setOption('solver', 'cbc')
ampl.solve()
print(ampl.solve_result)

ampl.eval('display Completed;')
percentallocated = ampl.eval('display PercentAllocated;')
ampl.eval('display amountOfOvertimeUsed;')
# Display the results or further post-processing
# ...


cbc 2.10.10:cbc 2.10.10: optimal solution; objective 1.98
0 simplex iterations
solved
Completed [*] :=
 1  0
 2  0
 3  0
 4  0
 5  0
 6  1
 7  0
 8  0
 9  1
10  0
11  0
12  0
;

PercentAllocated [1,*,*] (tr)
:    1   2   3   4   5   6   7   8   9  10  11  12    :=
1    0   0   0   0   0   0   0   0   0   0   0   0
2    0   0   0   0   0   0   0   0   0   0   0   0
3    0   0   0   0   0   0   0   0   0   0   0   0
4    0   0   0   0   0   0   0   0   0   0   0   0
5    0   0   0   0   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0   0   0   0   0   0
9    0   0   0   0   0   0   0   0   0   0   0   0
10   0   0   0   0   0   0   0   0   0   0   0   0
11   0   0   0   0   0   0   0   0   0   0   0   0
12   0   0   0   0   0   0   0   0   0   0   0   0
13   0   0   0   0   0   0   0   0   0   0   0   0

 [2,*,*] (tr)
:    1   2   3   4   5   6   7   8   9  10  11  12    :=


In [12]:

data = []
# extract data from ampl into a data list
for i in range(1, num_people+1):
    for j in range(1, num_projects+1):
        time = 0
        for k in range(1, num_skills+1):
            time  += ampl.getVariable('PercentAllocated').get(i, j,k).value()
        data.append({"Person": people[i-1]["Name"], "Project": projects[j-1]["Name"], "Assigned": time})


In [13]:


# Create a DataFrame for the assignment results

assignment_df = pd.DataFrame(data)

# Pivot the DataFrame to have projects as headers
assignment_pivot = assignment_df.pivot(index='Person', columns='Project', values='Assigned')
assignment_pivot.to_excel('results.xlsx')
# Write the DataFrame to an Excel file

file_path = filedialog.asksaveasfilename(defaultextension='.xlsx', filetypes=[('Excel files', '*.xlsx')])
# Create a tkinter root window (hidden)
root = tk.Tk()
root.withdraw()
# Check if the user canceled the dialog
if not file_path:
    print("File save canceled.")
else:
    # Write the DataFrame to the selected location
    assignment_pivot.to_excel(file_path)
    print(f"Excel file saved to: {file_path}")

Excel file saved to: C:/Users/Ewan/OneDrive - Texas A&M University/result_test_clearance_ultimate_secret.xlsx
